# 階層ベイズモデルを用いた商品評価予測システム
## 少数レビューから真の評価を予測するシステム

In [ ]:
import pymc as pm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import arviz as az
import pickle

# 日本語フォント設定
plt.rcParams['font.family'] = 'DejaVu Sans'
plt.rcParams['font.size'] = 12

## 1. デモ用データの作成
3つのカテゴリ（家電、書籍、アパレル）の商品レビューデータを作成

In [ ]:
# デモ用レビューデータの作成
np.random.seed(42)

# カテゴリ定義: 0=家電, 1=書籍, 2=アパレル
category_names = ['家電', '書籍', 'アパレル']

# 各商品のレビューデータ (1-5星評価)
reviews = np.array([
    # 商品A (家電) - レビュー多数
    5, 4, 5, 4, 4, 3, 5, 4,
    # 商品B (家電) - レビュー中程度  
    4, 4, 3, 5, 4,
    # 商品C (書籍) - レビュー多数
    5, 5, 4, 4, 3, 4, 5,
    # 商品D (書籍) - レビュー少数
    3, 4,
    # 商品E (アパレル) - レビュー1件のみ（予測対象）
    1,
    # 商品F (家電) - レビュー1件のみ（予測対象）
    5,
    # 商品G (アパレル) - レビュー少数
    2, 3, 2
])

# 各レビューがどの商品に属するかのインデックス
product_idx = np.array([0,0,0,0,0,0,0,0,  # 商品A (8件)
                       1,1,1,1,1,         # 商品B (5件)
                       2,2,2,2,2,2,2,     # 商品C (7件)
                       3,3,               # 商品D (2件)
                       4,                 # 商品E (1件)
                       5,                 # 商品F (1件)
                       6,6,6])            # 商品G (3件)

# 各商品が属するカテゴリ
category_of_product = np.array([0, 0, 1, 1, 2, 0, 2])  # A,B=家電, C,D=書籍, E,G=アパレル, F=家電
category_idx = category_of_product[product_idx]

num_products = len(np.unique(product_idx))
num_categories = len(np.unique(category_idx))

print(f"商品数: {num_products}")
print(f"カテゴリ数: {num_categories}")
print(f"総レビュー数: {len(reviews)}")

# データフレームで確認
product_names = ['商品A(家電)', '商品B(家電)', '商品C(書籍)', '商品D(書籍)', '商品E(アパレル)', '商品F(家電)', '商品G(アパレル)']
review_counts = [np.sum(product_idx == i) for i in range(num_products)]
avg_ratings = [np.mean(reviews[product_idx == i]) for i in range(num_products)]

df = pd.DataFrame({
    '商品名': product_names,
    'カテゴリ': [category_names[cat] for cat in category_of_product],
    'レビュー数': review_counts,
    '平均評価': [f"{avg:.2f}" for avg in avg_ratings]
})

print("\n=== 商品データ一覧 ===")
print(df)

## 2. 階層ベイズモデルの実装
2階層構造: カテゴリレベル → 個別商品レベル

In [ ]:
# 階層ベイズモデルの定義
with pm.Model() as hierarchical_model:
    # --- カテゴリレベル（上位階層）---
    # 各カテゴリの平均評価傾向（事前分布）
    mu_category = pm.Normal('mu_category', mu=3.5, sigma=1.0, shape=num_categories)
    
    # カテゴリ内のばらつき
    sigma_category = pm.HalfNormal('sigma_category', sigma=0.5)
    
    # --- 個別商品レベル（下位階層）---
    # 各商品の真の評価は、所属カテゴリの平均を中心とした分布に従う
    mu_product = pm.Normal('mu_product', 
                          mu=mu_category[category_of_product], 
                          sigma=sigma_category, 
                          shape=num_products)
    
    # --- 観測モデル ---
    # 1-5星評価を0-4にスケール変換してベータ二項分布でモデリング
    # mu_productを0-1の範囲に変換
    p_success = pm.math.sigmoid(mu_product[product_idx] - 2.5)  # 2.5を中心にシグモイド変換
    
    # ベータ二項分布のパラメータ
    n_trials = 4  # 1-5星 → 0-4の範囲
    alpha = p_success * 10  # 濃度パラメータ
    beta = (1 - p_success) * 10
    
    # 観測されたレビューをベータ二項分布でモデリング
    y_obs = pm.BetaBinomial('y_obs', 
                           n=n_trials, 
                           alpha=alpha, 
                           beta=beta, 
                           observed=reviews - 1)  # 1-5 → 0-4に変換
    
    # MCMCサンプリング
    trace = pm.sample(1000, tune=500, chains=2, cores=1, random_seed=42)

print("モデル学習完了！")

## 3. 予測結果の分析

In [ ]:
# 結果のサマリー
summary = az.summary(trace, var_names=['mu_category', 'mu_product'], hdi_prob=0.95)
print("=== カテゴリ別平均評価の推定結果 ===")
for i, cat_name in enumerate(category_names):
    mean_val = summary.loc[f'mu_category[{i}]', 'mean']
    hdi_lower = summary.loc[f'mu_category[{i}]', 'hdi_2.5%']
    hdi_upper = summary.loc[f'mu_category[{i}]', 'hdi_97.5%']
    print(f"{cat_name}: {mean_val:.2f} (95%区間: {hdi_lower:.2f} - {hdi_upper:.2f})")

print("\n=== 商品別評価予測結果 ===")
for i in range(num_products):
    mean_val = summary.loc[f'mu_product[{i}]', 'mean']
    hdi_lower = summary.loc[f'mu_product[{i}]', 'hdi_2.5%']
    hdi_upper = summary.loc[f'mu_product[{i}]', 'hdi_97.5%']
    actual_avg = avg_ratings[i]
    review_count = review_counts[i]
    
    print(f"{product_names[i]} (レビュー{review_count}件):")
    print(f"  実際の平均: {actual_avg:.2f}")
    print(f"  予測評価: {mean_val:.2f} (95%区間: {hdi_lower:.2f} - {hdi_upper:.2f})")
    
    if review_count <= 2:  # レビューが少ない商品をハイライト
        print(f"  ★ 少数レビュー商品: 縮退効果により予測が安定化")
    print()

## 4. 予測結果の可視化

In [ ]:
# 予測結果の可視化
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# 左：カテゴリ別平均評価
category_means = [summary.loc[f'mu_category[{i}]', 'mean'] for i in range(num_categories)]
category_lower = [summary.loc[f'mu_category[{i}]', 'hdi_2.5%'] for i in range(num_categories)]
category_upper = [summary.loc[f'mu_category[{i}]', 'hdi_97.5%'] for i in range(num_categories)]

ax1.errorbar(range(num_categories), category_means, 
            yerr=[np.array(category_means) - np.array(category_lower),
                  np.array(category_upper) - np.array(category_means)],
            fmt='o', capsize=5, capthick=2, markersize=8)
ax1.set_xticks(range(num_categories))
ax1.set_xticklabels(category_names)
ax1.set_ylabel('予測評価')
ax1.set_title('カテゴリ別平均評価の推定')
ax1.grid(True, alpha=0.3)
ax1.set_ylim(1, 5)

# 右：商品別評価（実測値 vs 予測値）
product_means = [summary.loc[f'mu_product[{i}]', 'mean'] for i in range(num_products)]
product_lower = [summary.loc[f'mu_product[{i}]', 'hdi_2.5%'] for i in range(num_products)]
product_upper = [summary.loc[f'mu_product[{i}]', 'hdi_97.5%'] for i in range(num_products)]

# レビュー数による色分け
colors = ['red' if count <= 2 else 'blue' for count in review_counts]

# 実測値
ax2.scatter(range(num_products), avg_ratings, color='green', s=100, alpha=0.7, label='実際の平均評価')

# 予測値
ax2.errorbar(range(num_products), product_means,
            yerr=[np.array(product_means) - np.array(product_lower),
                  np.array(product_upper) - np.array(product_means)],
            fmt='o', capsize=3, capthick=2, markersize=6, 
            color='red', alpha=0.8, label='階層ベイズ予測')

ax2.set_xticks(range(num_products))
ax2.set_xticklabels([name.replace('商品', '') for name in product_names], rotation=45)
ax2.set_ylabel('評価値')
ax2.set_title('商品別評価予測 (赤: レビュー少数商品)')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(1, 5)

plt.tight_layout()
plt.show()

print("図の解釈:")
print("- 左図: カテゴリごとの評価傾向を学習")
print("- 右図: 緑=実測値、赤=階層ベイズ予測値")
print("- レビューが少ない商品(E,F)では、カテゴリ平均に引っ張られる縮退効果を確認")

## 5. 縮退効果の確認
レビューが少ない商品の予測がどの程度安定化されるかを確認

In [ ]:
# 縮退効果の詳細分析
print("=== 縮退効果の分析 ===")
print("\n▼ レビューが1件のみの商品の予測結果:")

# 商品E (アパレル, レビュー1件: 星1)
product_e_idx = 4
actual_rating_e = 1.0
predicted_rating_e = summary.loc[f'mu_product[{product_e_idx}]', 'mean']
category_avg_apparel = summary.loc['mu_category[2]', 'mean']  # アパレルカテゴリ

print(f"商品E (アパレル):")
print(f"  元レビュー: 星{actual_rating_e}")
print(f"  予測評価: {predicted_rating_e:.2f}")
print(f"  アパレルカテゴリ平均: {category_avg_apparel:.2f}")
print(f"  縮退効果: {predicted_rating_e - actual_rating_e:+.2f} (カテゴリ平均に引っ張られて上昇)")

# 商品F (家電, レビュー1件: 星5)
product_f_idx = 5
actual_rating_f = 5.0
predicted_rating_f = summary.loc[f'mu_product[{product_f_idx}]', 'mean']
category_avg_electronics = summary.loc['mu_category[0]', 'mean']  # 家電カテゴリ

print(f"\n商品F (家電):")
print(f"  元レビュー: 星{actual_rating_f}")
print(f"  予測評価: {predicted_rating_f:.2f}")
print(f"  家電カテゴリ平均: {category_avg_electronics:.2f}")
print(f"  縮退効果: {predicted_rating_f - actual_rating_f:+.2f} (カテゴリ平均に引っ張られて下降)")

print(f"\n▼ 階層ベイズの効果:")
print(f"- 極端な評価(星1, 星5)が、より現実的な値に調整される")
print(f"- カテゴリの知識を活用して、より信頼性の高い予測を実現")
print(f"- レビュー数が多い商品(A,B,C)は実測値に近い予測値")

## 6. モデルの保存

In [ ]:
# モデルパラメータの保存
model_results = {
    'trace': trace,
    'model': hierarchical_model,
    'summary': summary,
    'category_names': category_names,
    'product_names': product_names,
    'num_products': num_products,
    'num_categories': num_categories
}

# pickle形式で保存
with open('hierarchical_bayes_model.pkl', 'wb') as f:
    pickle.dump(model_results, f)

print("モデルを 'hierarchical_bayes_model.pkl' に保存しました")
print("\n=== プロジェクト実装完了 ===")
print("✓ 2階層ベイズモデルによる商品評価予測システム")
print("✓ 少数レビュー商品の安定化予測")
print("✓ カテゴリ知識を活用した縮退効果")
print("✓ 95%信頼区間による不確実性の定量化")